# Migrating from `azureml-core` Tracking to `MLflow` Tracking APIs

## Goal

The goals of this notebook are:
1. To provide a comparison of logging APIs in `azureml-core` and `MLflow`
2. To provide examples that can be executed in real-time.
3. Provide a way to migrate from `azureml-core`'s run management and tracking APIs to `MLflow` run management and tracking APIs

## Prerequisites

Before proceeding, please make sure you have the following pip packages installed:
1. `azureml-core`
2. `mlflow`
3. `azureml-mlflow`

Please make sure you also have an Azure Machine Learning workspace. You can create one using the following steps: https://docs.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources

## Relevant Documentation

If you would like a more detailed view of the logging APIs available in `MLflow`, please see [here](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_artifact). For a general overview of `MLflow` tracking in general, please see [here](https://www.mlflow.org/docs/latest/tracking.html).

## Setup

Run the following cells to retrieve your workspace and set the `MLflow` tracking URI to point at the AzureML backend. This step is required for `MLflow` metrics and artifacts to get logged properly to your workspace.

In [ ]:
%pip install -r requirements.txt

In [ ]:
from azureml.core import Workspace
import mlflow

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

## Creating and Managing Experiments with `MlFlow`

You can create and manage experiments using `Mlflow` just as you can with `azureml-core`. You can find more information on `mlflow.create_experiment()` [here](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.create_experiment), `mlflow.get_experiment_by_name()` [here]("https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.get_experiment"), and `mlflow.list_experiments()` [here](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.list_experiments). You can also set the default experiment to use for all runs with `mlflow.set_experiment()`; more information can be found [here](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.set_experiment).

In [ ]:
from mlflow.tracking import MlflowClient

mlflow.create_experiment("create-experiment-mlflow")
print(mlflow.get_experiment_by_name("create-experiment-mlflow"))
print(mlflow.list_experiments())

## Creating and Managing Runs with `MLflow`

### Interactive Runs

In `azureml-core`, interactive runs are started using `experiment.start_logging()`. You can start interactive runs using `MLflow`and an `AzureML` backend by doing the following:

1. Follow the setup instructions above
1. If you want to set an experiment for the run, you can call `mlflow.set_experiment("<name of experiment>")`, which will automatically create any new runs under that experiment.
1. Start the run by either:
    1. Using `mlflow.start_run()`. This method returns an [`mlflow.ActiveRun`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.ActiveRun) (clickable link).
        2. You can call this method using the context manager paradigm: `with mlflow.start_run() as run`.
    1. Calling any of `MLflow`'s logging APIs. A new run will be started if one does not exist. You can retrieve the current active run by calling `mlflow.active_run()`
1. Complete the run. 
    1. If you are using the run as a context manager, the run will automatically complete when the context manager exits.
    1. Otherwise, you can end the currently active run by calling `mlflow.end_run()`


**Note**: The `mlflow.ActiveRun` object returned by `mlflow.active_run()` **will not** contain items like parameters, metrics, etc. You may find more information [here](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.active_run). See the section `Viewing Run Metrics` to see the steps retrieving metrics for a run

In [ ]:
# Starting an MLflow run using a context manager
mlflow.set_experiment("create-experiment-mlflow-context-manager")
with mlflow.start_run() as run:
    # run is started when context manager is entered and ended when context manager exits
    pass

In [ ]:
# Starting an MLflow
mlflow.set_experiment("create-experiment-mlflow-manual")
interactive_mlflow_run = mlflow.start_run()
# End the MLflow run manually
mlflow.end_run()

In [ ]:
# Finally, an MLflow run can be started by calling one of the logging APIs
mlflow.set_experiment("create-experiment-mlflow-logging")
mlflow.log_metric("sample_metric", 1)
# Note that the run object returned by `mlflow.active_run()` will not have data like metrics and parameters
run = mlflow.active_run()
print(run)
# End the MLflow run manually
mlflow.end_run()

### Remote Runs

For remote training runs, the tracking URI and experiment for the run are already set by the AzureML backend. Instead of calling `azureml-core`'s `Run.get_context()` to retrieve the run inside of a training script, you can either:
1. Call `mlflow.start_run()` to get the `MLflow` run for that particular training job. As with an interactive local run, you can use this method with the context manager paradigm.
1. Or, use one of the `MLflow` logging APIs to start logging metrics and artifacts. Afterwards, you can call `mlflow.active_run()` to retrieve the current `MLflow` run.

It is important to note that unless you do one of the above in your training script, `mlflow.active_run()` will return `None`.

## Logging API Comparison

The following section outlines each of the logging APIs available in the `azureml-core` and presents how to use `MLflow` to either:
1. Reproduce the same behavior
2. Provide functionality similar to that of `azureml-core`

The structure of the document will be as follows:

## AzureML Tracking API

< Table describing AzureML API, accepted parameters, and available `MLflow` alternatives with notes. The `MLflow` alternatives will have clickable links to the official `MLflow` documentation for that method >

< Code examples >

### Start the interactive `AzureML` Run

In [ ]:
from azureml.core import Experiment

# create an AzureML experiment and start a run
experiment = Experiment(ws, "logging-with-mlflow-and-azureml-core")
azureml_run = experiment.start_logging()

### Start the interactive `MLflow` Run

In [ ]:
# Set the MLflow experiment and start a run
mlflow.set_experiment("logging-with-mlflow-and-azureml-core")
mlflow_run = mlflow.start_run()

### 

## `azureml.core.Run.log()`

| azureml.core API                   | MLFlow API                                | Notes |
|-------------------------------|-------------------------------------------|-----------------|
| `Run.log(name: str, value: float, description="", step=None)` | [`mlflow.log_metric(key: str, value: float, step: Optional[int] = None)`](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_metric)                                |    |
| `Run.log(self, name: str, value: bool, description="", step=None)` |  [`mlflow.log_metric(key: str, value: float, step: Optional[int] = None)`](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_metric), where `value` is 0 or 1 |  |
| `Run.log(self, name: str, value: str, description="", step=None)` | `mlflow.log_text(text: str, artifact_file: str) ` | `mlflow.log_text()` logs the string as an artifact and will not be considered a metric. The result of `log_text()` will be displayed in the `Outputs + logs` tab of the AzureML Studio UI  |

### Code example: logging an integer or float metric

In [ ]:
# Using AzureML
azureml_run.log("sample_int_metric", 1)

# Using MLflow
mlflow.log_metric("sample_int_metric", 1)

### Code example: logging a boolean metric

In [ ]:
# Using AzureML
azureml_run.log("sample_boolean_metric", True)

# Using MLflow
mlflow.log_metric("sample_boolean_metric", 1)

### Code example: logging a string metric

In [ ]:
# Using AzureML
azureml_run.log("sample_string_metric", "a_metric")

# Using MLflow. Note that the string will get logged only as an artifact and will not be logged as a metric
mlflow.log_text("sample_string_text", "string.txt")

## `azureml.core.Run.log_image()`

| azureml.core API                    | MLFlow equivalent or proposed alternative | Notes |
|-------------------------------|-------------------------------------------|-------|
| `Run.log_image(name: str, plot: Optional[matplotlib.pyplot] = None)`  | [`mlflow.log_figure(figure: Union[matplotlib.figure.Figure, plotly.graph_objects.Figure], artifact_file: str)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_figure)  |  <ul><li/>Appears in Images tab.<li/>Logged as an artifact<li/>Both appear in Studio UI<li/>`mlflow.log_figure` is **experimental**</ul>                          
| `Run.log_image(name: str, path: Optional[Union[str, os.PathLike, pathlib.Path] = None)`  | [`mlflow.log_artifact(local_path: str, artifact_path: Optional[str] = None)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_artifact) |<ul><li/>Image logged with `MLflow` appears in the Images tab.<li/>`MLflow` logs image as an artifact<li/>Works with png and jpeg.</ul>       |

### Logging an image saved to a png or jpeg file

In [ ]:
# Using AzureML
azureml_run.log_image("sample_image", path="Azure.png")

# Using MLflow
mlflow.log_artifact("Azure.png")

### Logging a matplotlib.pyplot

In [ ]:
import matplotlib.pyplot as plt

plt.plot([1, 2, 3])

# Using AzureML
azureml_run.log_image("sample_pyplot", plot=plt)

# Using MLflow
fig, ax = plt.subplots()
ax.plot([0, 1], [2, 3])
mlflow.log_figure(fig, "sample_pyplot.png")

## `azureml.core.Run.log_list()`

| azureml.core API              | MLFlow equivalent or proposed alternative |       Notes     |
|-------------------------------|-------------------------------------------|-----------------|
| `Run.log_list(name, value, description="")`   | [`MlFlowClient().log_batch(run_id: str, metrics: Sequence[Metric] = (), params: Sequence[Param] = (), tags: Sequence[RunTag] = ())`](https://mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient.log_batch) | <ul><li/>Metrics logged with `MLflow` in metrics tab.<li/>`MLflow` does not support logging text metrics</ul>       |

In [ ]:
list_to_log = [1, 2, 3, 2, 1, 2, 3, 2, 1]

# Using AzureML
azureml_run.log_list("sample_list", list_to_log)

# Using MLflow
from mlflow.entities import Metric
from mlflow.tracking import MlflowClient
import time

metrics = [
    Metric(key="sample_list", value=val, timestamp=int(time.time() * 1000), step=0)
    for val in list_to_log
]
MlflowClient().log_batch(mlflow_run.info.run_id, metrics=metrics)

## `azureml.core.Run.log_row()`

| azureml.core API                    | MLFlow equivalent or proposed alternative | Notes |
|-------------------------------|-------------------------------------------|-------|
| `Run.log_row(self, name, description=None, **kwargs)` | [`mlflow.log_metrics(metrics: Dict[str, float], step: Optional[int] = None)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_metrics) | <ul><li/>Does not render in UI as a table.<li/>Does not support logging text values</li></ul> |

In [ ]:
# v1 SDK
azureml_run.log_row("sample_table", col1=5, col2=10)

# MLFlow
metrics = {"sample_table.col1": 5, "sample_table.col2": 10}
mlflow.log_metrics(metrics)

## `azureml.core.Run.log_table()`

| azureml.core API                    | MLFlow equivalent or proposed alternative | Notes           |
|-------------------------------|-------------------------------------------|-----------------|
| `Run.log_table(name, value, description="")` | [`mlflow.log_metrics(metrics: Dict[str, float], step: Optional[int] = None)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_metrics)                               | <ul><li/>`MLflow`logs metrics as different metrics for each column<li/>Metrics logged with `MLflow` appear in the metrics tab but not as a table<li/>`MLflow` does not support logging text values.</ul>        |
| `Run.log_table(name, value, description="")` | [`mlflow.log_artifact(local_path: str, artifact_path: Optional[str] = None)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_artifact) | <ul><li/>Stored as an artifact<li>Does not appear in the metrics.</ul>        |

In [ ]:
# Using AzureML
table = {"col1": [1, 2, 3], "col2": [4, 5, 6]}
azureml_run.log_table("table", table)

# Using mlflow.log_metrics
# Add a metric for each column prefixed by metric name. Similar to log_row
row1 = {"table.col1": 5, "table.col2": 10}
# To be done for each row in the table
mlflow.log_metrics(row1)

# Using mlflow.log_artifact
import json

with open("table.json", "w") as f:
    json.dump(table, f)
mlflow.log_artifact("table.json")

## `azureml.core.Run.log_accuracy_table()`

| azureml.core API              | MLFlow equivalent or proposed alternative |  Notes          |
|-------------------------------|-------------------------------------------|-----------------|
| `Run.log_accuracy_table(name, value, description="")`   | [`mlflow.log_dict(dictionary: Any, artifact_file: str)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_dict)| <ul><li/>Dict logged with `MLflow` does not render in the Studio UI as accuracy table.<li/>Dict logged with `MLflow` does not appear in the metrics tab<li/>`MLflow` logs dict as an artifact<li/>`log_dict` is **experimental**</ul>        |

In [ ]:
ACCURACY_TABLE = (
    '{"schema_type": "accuracy_table", "schema_version": "v1", "data": {"probability_tables": '
    + "[[[114311, 385689, 0, 0], [0, 0, 385689, 114311]], [[67998, 432002, 0, 0], [0, 0, "
    + '432002, 67998]]], "percentile_tables": [[[114311, 385689, 0, 0], [1, 0, 385689, '
    + '114310]], [[67998, 432002, 0, 0], [1, 0, 432002, 67997]]], "class_labels": ["0", "1"], '
    + '"probability_thresholds": [0.52], "percentile_thresholds": [0.09]}}'
)

# Using AzureML
azureml_run.log_accuracy_table("v1_accuracy_table", ACCURACY_TABLE)

# Using MLflow
mlflow.log_dict(ACCURACY_TABLE, "mlflow_accuracy_table.json")

## `azureml.core.Run.log_confusion_matrix()`

| azureml.core API              | MLFlow equivalent or proposed alternative |  Notes          |
|-------------------------------|-------------------------------------------|-----------------|
| `Run.log_confusion_matrix(name, value, description="")`   | [`mlflow.log_dict(dictionary: Any, artifact_file: str)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_dict)| <ul><li/>Dict logged with `MLflow` does not render in the Studio UI as confusion matrix.<li/>Dict logged with `MLflow` does not appear in the metrics tab<li/>`MLflow` logs dict as an artifact<li/>`log_dict` is **experimental**</ul>        |

In [ ]:
CONF_MATRIX = (
    '{"schema_type": "confusion_matrix", "schema_version": "v1", "data": {"class_labels": '
    + '["0", "1", "2", "3"], "matrix": [[3, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]]}}'
)

# v1 SDK
azureml_run.log_confusion_matrix("v1_confusion_matrix", json.loads(CONF_MATRIX))

# MLFlow
mlflow.log_dict(CONF_MATRIX, "mlflow_confusion_matrix.json")

## `azureml.core.Run.log_predictions()`

| azureml.core API              | MLFlow equivalent or proposed alternative |  Notes          |
|-------------------------------|-------------------------------------------|-----------------|
| `Run.log_predictions(name, value, description="")`   | [`mlflow.log_dict(dictionary: Any, artifact_file: str)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_dict)| <ul><li/>Dict logged with `MLflow` does not render in the Studio UI as predictions.<li/>Dict logged with `MLflow` does not appear in the metrics tab<li/>`MLflow` logs dict as an artifact<li/>`log_dict` is **experimental**</ul>        |

In [ ]:
PREDICTIONS = (
    '{"schema_type": "predictions", "schema_version": "v1", "data": {"bin_averages": [0.25,'
    + ' 0.75], "bin_errors": [0.013, 0.042], "bin_counts": [56, 34], "bin_edges": [0.0, 0.5, 1.0]}}'
)

# v1 SDK
azureml_run.log_predictions("test_predictions", json.loads(PREDICTIONS))

# MLFlow
mlflow.log_dict(PREDICTIONS, "mlflow_predictions.json")

## `azureml.core.Run.log_residuals()`

| azureml.core API              | MLFlow equivalent or proposed alternative |  Notes          |
|-------------------------------|-------------------------------------------|-----------------|
| `Run.log_residuals(name, value, description="")`   | [`mlflow.log_dict(dictionary: Any, artifact_file: str)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_dict)| <ul><li/>Dict logged with `MLflow` does not render in the Studio UI as residuals.<li/>Dict logged with `MLflow` does not appear in the metrics tab<li/>`MLflow` logs dict as an artifact<li/>`log_dict` is **experimental**</ul>        |

In [ ]:
RESIDUALS = (
    '{"schema_type": "residuals", "schema_version": "v1", "data": {"bin_edges": [100, 200, 300], '
    + '"bin_counts": [0.88, 20, 30, 50.99]}}'
)

# v1 SDK
azureml_run.log_residuals("test_residuals", json.loads(RESIDUALS))

# MLFlow
mlflow.log_dict(RESIDUALS, "mlflow_residuals.json")

In [ ]:
# End the AzureML and MLflow runs
azureml_run.complete()
mlflow.end_run()

## Retreiving Run Info and Data with `MlFlow`

You can access run information using `MLflow` through the run object's `data` and `info` properties. See [here](https://mlflow.org/docs/latest/python_api/mlflow.entities.html#mlflow.entities.Run) for more information on the `MLflow.entities.Run` object and the information it exposes.

After run has completed, you can retrieve it using the [`MlFlowClient()`](https://mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient) (clickable link). 

In [ ]:
from mlflow.tracking import MlflowClient

# Use MlFlow to retrieve the run that was just completed
client = MlflowClient()
finished_mlflow_run = MlflowClient().get_run(mlflow_run.info.run_id)

You can view the metrics, parameters, and tags for the run in the `data` field of the run object.

In [ ]:
metrics = finished_mlflow_run.data.metrics
tags = finished_mlflow_run.data.tags
params = finished_mlflow_run.data.params

**Note:** The metrics dictionary under `mlflow.entities.Run.data.metrics` will only have the **most recently logged value** for a given metric name. For example, if you log, in order, 1, then 2, then 3, then 4 to a metric called `sample_metric`, only 4 will be present in the metrics dictionary for the key `sample_metric`.

To get all metrics logged for a particular metric name, you can use `MlFlowClient.get_metric_history()`.

In [ ]:
with mlflow.start_run() as multiple_metrics_run:
    mlflow.log_metric("sample_metric", 1)
    mlflow.log_metric("sample_metric", 2)
    mlflow.log_metric("sample_metric", 3)
    mlflow.log_metric("sample_metric", 4)

print(client.get_run(multiple_metrics_run.info.run_id).data.metrics)
print(client.get_metric_history(multiple_metrics_run.info.run_id, "sample_metric"))

You can view general information about the run, such as start time, run id, experiment id, etc. through the `info` field of the run object

In [ ]:
run_start_time = finished_mlflow_run.info.start_time
run_experiment_id = finished_mlflow_run.info.experiment_id
run_id = finished_mlflow_run.info.run_id

## Retrieving Run Artifacts with `MLflow`

To view the artifacts of a run, you can use [`MlFlowClient.list_artifacts()`](https://mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient.list_artifacts) (clickable link)

In [ ]:
client.list_artifacts(finished_mlflow_run.info.run_id)

To download an artifact, you can use [`MLFlowClient.download_artifacts()`](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient.download_artifacts) (clickable link)

In [ ]:
client.download_artifacts(finished_mlflow_run.info.run_id, "Azure.png")

## Searching Runs with `MLflow`

You can use [`mlflow.search_runs()`](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.search_runs) (clickable link) to query for runs programatically. The search API is a simplified
version of the SQL WHERE clause and supports the following functionality:
1. query on metrics, params
1. query on tags
1. query on run metadata (for example, status)
1. query on runs of single experiment or multiple experiments

The `search_runs()` call returns either:
1. By default, a `pandas.DataFrame`.
1. Optionally, a list. You can specify the list option by passing `"list"` as the value for the keyword argument `output_format`.


In [ ]:
from mlflow.entities import ViewType

## example: get list of runs in order of descending accuracy and return as a list, use:
runs = mlflow.search_runs(
    experiment_ids="0",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.acc DESC"],
    output_format="list",
)

In [ ]:
## example: get all active runs from experiments IDs 3, 4, and 17 that used a CNN model with 10 layers
query = "params.model = 'CNN' and params.layers = '10'"
runs = mlflow.search_runs(
    experiment_ids=["3", "4", "17"],
    filter_string=query,
    run_view_type=ViewType.ACTIVE_ONLY,
)